In [1]:
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd Lifespan_Age_Transformation_Synthesis/
!pip3 install -r requirements.txt

Cloning into 'Lifespan_Age_Transformation_Synthesis'...
remote: Enumerating objects: 604, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 604 (delta 51), reused 20 (delta 9), pack-reused 512
Receiving objects: 100% (604/604), 22.85 MiB | 23.08 MiB/s, done.
Resolving deltas: 100% (370/370), done.
/content/Lifespan_Age_Transformation_Synthesis
     |████████████████████████████████| 686kB 7.4MB/s 
     |████████████████████████████████| 245kB 14.6MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp37-none-any.whl size=655251 sha256=4144ac01b98f519e7c4bd1e4e8e190d7a329a875babafca21bbb4b298064646c
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=e7cb180a647f3d6d4056dcfc48bcbcaa0173d37adc307d421a07f4646887f9f7
  Store

In [2]:
!python download_models.py

100% 213M/213M [00:06<00:00, 30.7MB/s]
Extracting males model zip file
Done!
100% 213M/213M [00:07<00:00, 28.5MB/s]
Extracting females model zip file
Done!
100% 99.7M/99.7M [00:07<00:00, 13.4MB/s]
Done!
100% 178M/178M [00:06<00:00, 26.3MB/s]
Done!
100% 464M/464M [00:09<00:00, 47.3MB/s]
Done!


In [8]:
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

In [8]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)

In [8]:
opt.name = 'males_model' # change to 'females_model' if you're trying the code on a female image
model = create_model(opt)
model.eval()

In [6]:
# upload your image (the code supports only a single image at a time)
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  img_path = filename
  print('User uploaded file "{name}"'.format(name=filename))

Saving Untitled design.png to Untitled design.png
User uploaded file "Untitled design.png"


In [8]:
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

In [8]:
use_webm = False
# For some unknown reason the mp4 video is not displayed on chrome
# If you have chrome, uncomment the following lines to convert the 
# result to webm for display purposes

# !pip3 install webm
# webm_out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.webm')
# !webm -i $out_path $webm_out_path
# use_webm = True

from IPython.display import HTML
from base64 import b64encode
video_path = webm_out_path if use_webm else out_path
video_type = "video/webm" if use_webm else "video/mp4"
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width={0} controls>
      <source src="{1}" type="{2}">
</video>
""".format(opt.fineSize, data_url, video_type))

In [ ]:
files.download(out_path)